In [1]:
#Libraries

#Python Libs
import sys
import os
import glob
import traceback
from datetime import datetime
import time
from geopy import distance


#Data Analysis Libs
import pandas as pd
import numpy as np


In [2]:
#Functions
def select_input_files(enh_buste_base_path,init_date,fin_date,suffix):
        selected_files = []
        all_files = glob.glob(os.path.join(enh_buste_base_path,"*"))

        for file_ in all_files:
                try:
                        file_date = pd.to_datetime(file_.split('/')[-1],format=('%Y_%m_%d' + suffix  + '.csv'))
                        if (file_date >= init_date) and (file_date <= fin_date):
                                selected_files.append((file_,file_date))
                except:
                        continue

        return sorted(selected_files)

def dist(p1_lat, p1_lon, p2_lat, p2_lon):
    return np.around(distance.geodesic((p1_lat,p1_lon),(p2_lat,p2_lon)).km,decimals=5)

def get_router_id(query_date):
    INTERMEDIATE_OTP_DATE = pd.to_datetime("2017-06-30", format="%Y-%m-%d")
    router_id = ''

    if (query_date <= INTERMEDIATE_OTP_DATE):
        return 'ctba-2017-1'
    else:
        return 'ctba-2017-2'

In [3]:
def choose_leg_matches(leg_matches_groups):
        chosen_leg_matches = pd.DataFrame(columns = otp_legs_buste.columns.values)
        prev_group_id = ()
        prev_leg_end_time = pd.NaT
        num_groups_not_survived = 0

        for name, group in legs_matches_groups:

                if (prev_group_id != name[0:2]):
                        prev_leg_end_time = otp_suggestions['date'][0]

                print
                print prev_leg_end_time
                print
                print "Original Group"
                print group.filter(['otp_start_time','matched_start_time'])

                if (group['mode'].iloc[0] == 'WALK'):
                    filtered_group = group
                    filtered_group['matched_end_time'] = prev_leg_end_time + filtered_group['otp_duration_mins']
                else:
                    filtered_group = group[group['matched_start_time'] > prev_leg_end_time]
                print
                print "Filtered Group"
                print filtered_group.filter(['otp_start_time','matched_start_time'])

                if (len(filtered_group) == 0):
                        num_groups_not_survived += 1
                        continue

                chosen_leg_match = filtered_group.sort_values('boarding_matched_start_timediff').iloc[0]
                print "Chosen Leg"
                print chosen_leg_match

                chosen_leg_matches = chosen_leg_matches.append(chosen_leg_match)

                                #Update variables
                prev_group_id = name[0:2]
                prev_leg_end_time = chosen_leg_match['matched_end_time']

                chosen_leg_matches = chosen_leg_matches.filter(otp_legs_buste.columns.values) \
                                                        .assign(user_trip_id = lambda x: x['user_trip_id'].astype(int),
                                                                itinerary_id = lambda x: x['itinerary_id'].astype(int),
                                                                leg_id = lambda x: x['leg_id'].astype(int))


                #print num_groups_not_survived
                return chosen_leg_matches


In [4]:
def prepare_otp_data(otp_data):
        otp_data['otp_start_time'] = otp_data['otp_start_time'] - pd.Timedelta('10800 s')
        otp_data['otp_end_time'] = otp_data['otp_end_time'] - pd.Timedelta('10800 s')
        otp_data['route'] = otp_data['route'].astype(str)
        otp_data['route'] = np.where(otp_data['mode'] == 'BUS',
                            otp_data['route'].astype(str).str.replace("\.0",'').str.zfill(3),
                            otp_data['route'])

        return otp_data

def compatible_dates(otp_data,ticketing_data):
        otp_date = otp_data['date'].iloc[0]
        ticketing_date = pd.to_datetime(ticketing_data['o_boarding_datetime'].dt.strftime('%Y-%m-%d')[0])

        return (otp_date == ticketing_date,otp_date,ticketing_date)


In [5]:
def match_vehicle_boardings(selected_trips,itineraries_start):
        vehicle_boarding_origins = selected_trips[np.logical_not(selected_trips['o_busCode'].str.isdigit())]
        matched_vehicle_boardings = vehicle_boarding_origins.merge(itineraries_start, left_on=['o_boarding_id','o_route','o_stopPointId'], right_on=['user_trip_id','route','from_stop_id'], how='inner')
        num_matched_vehicle_boardings = len(matched_vehicle_boardings.drop_duplicates(subset=['cardNum','o_boarding_id']))
        if num_matched_vehicle_boardings == 0:
            match_perc = 0.0
        else:
            match_perc = 100*(num_matched_vehicle_boardings/float(len(vehicle_boarding_origins)))
        return (matched_vehicle_boardings,num_matched_vehicle_boardings,match_perc)

def match_terminal_boardings(selected_trips,itineraries_start):
        terminal_boarding_origins = selected_trips[(selected_trips['o_busCode'].str.isdigit()) & (selected_trips['o_route'] != '021')]
        matched_terminal_boardings = terminal_boarding_origins.merge(itineraries_start,
                                                                                                                                left_on=['o_boarding_id','o_stopPointId'],
                                                                                                                                right_on=['user_trip_id','from_parent_station'], how='inner')
        num_matched_terminal_boardings = len(matched_terminal_boardings.drop_duplicates(subset=['cardNum','o_boarding_id']))
        if num_matched_terminal_boardings == 0:
            matched_perc = 0.0
        else:
            matched_perc = 100*(num_matched_terminal_boardings/float(len(terminal_boarding_origins)))
        return (matched_terminal_boardings,num_matched_terminal_boardings,matched_perc)

def match_terminal_021_boardings(selected_trips,itineraries_start):
        terminal_021_origins = selected_trips[(selected_trips['o_busCode'].str.isdigit()) & (selected_trips['o_route'] == '021')]
        matched_021_terminal_boardings = terminal_021_origins.merge(itineraries_start, left_on=['o_boarding_id','o_route','o_stopPointId'], right_on=['user_trip_id','route','from_parent_station'], how='inner')
        num_matched_021_terminal_boardings = len(matched_021_terminal_boardings.drop_duplicates(subset=['cardNum','o_boarding_id']))
        if num_matched_021_terminal_boardings == 0:
                terminal_021_match_perc = 0.0
        else:
                terminal_021_match_perc = 100*(num_matched_021_terminal_boardings/float(len(terminal_021_origins)))
        return (len(terminal_021_origins),matched_021_terminal_boardings,num_matched_021_terminal_boardings,terminal_021_match_perc)


In [9]:
def add_stops_data_to_leg_matches(chosen_leg_matches,stops_locations):
        chosen_leg_matches_data = chosen_leg_matches.merge(stops_locations, left_on='from_stop_id', right_on='stop_id', how='left') \
                                                                                .drop('stop_id', axis=1) \
                                                                                .rename(index=str, columns={'stop_lat':'from_stop_lat','stop_lon':'from_stop_lon'}) \
                                                                                .merge(stops_locations, left_on='to_stop_id', right_on='stop_id', how='left') \
                                                                                .drop('stop_id', axis=1) \
                                                                                .rename(index=str, columns={'stop_lat':'to_stop_lat','stop_lon':'to_stop_lon'}) \
                                                                                .merge(user_trips_ids, on=['user_trip_id'], how='inner') \
                                                                                [np.append(np.append(['cardNum'],otp_legs_buste.columns.values),['from_stop_lat','from_stop_lon','to_stop_lat','to_stop_lon'])]
        return chosen_leg_matches_data


In [10]:
def build_candidate_itineraries_df(chosen_leg_matches_data):
        otp_buste_itineraries = chosen_leg_matches_data \
                                        .groupby(['cardNum','user_trip_id','itinerary_id']) \
                                        .agg({'from_stop_id': lambda x: x.iloc[0],
                                                  'matched_start_time': lambda x: x.iloc[0],
                                                  'from_stop_lat': lambda x: x.iloc[0],
                                                  'from_stop_lon': lambda x: x.iloc[0],
                                                  'to_stop_id': lambda x: x.iloc[-1],
                                                  'matched_end_time': lambda x: x.iloc[-1],
                                                  'to_stop_lat': lambda x: x.iloc[-1],
                                                  'to_stop_lon': lambda x: x.iloc[-1],
                                                  'leg_id': lambda x: len(x),
                                                  'first_vehicle_boarding' : lambda x: x.any()}) \
                                        .rename(index=str, columns={'leg_id':'num_transfers','first_vehicle_boarding':'vehicle_boarding'}) \
                                        .add_prefix('match_') \
                                        .reset_index() \
                                        .assign(cardNum = lambda x: x['cardNum'].astype(float),
                                                        user_trip_id = lambda x: x['user_trip_id'].astype(int),
                                                        itinerary_id = lambda x: x['itinerary_id'].astype(int))
        return otp_buste_itineraries


In [11]:
def get_candidate_itineraries_summary(candidate_itineraries,boarding_suggestions_matches):
        otp_buste_itineraries_summary = candidate_itineraries \
                                        .merge(boarding_suggestions_matches \
                                                                .drop_duplicates(subset=['cardNum','user_trip_id','itinerary_id']),
                                                        on=['cardNum','user_trip_id','itinerary_id'],
                                                        how='inner') \
                                        [['cardNum', 'user_trip_id', 'itinerary_id',
                                                  'match_from_stop_id', 'match_matched_start_time', 'o_boarding_datetime',
                                                  'match_from_stop_lat', 'match_from_stop_lon', 'o_stop_lat', 'o_stop_lon',
                                                  'match_to_stop_id', 'match_matched_end_time', 'next_o_boarding_datetime',
                                                  'match_to_stop_lat', 'match_to_stop_lon', 'next_o_stop_lat', 'next_o_stop_lon','match_num_transfers', 'match_vehicle_boarding']] \
                                        .assign(start_diff = lambda x: np.absolute(x['match_matched_start_time'] - x['o_boarding_datetime']),
                                                        trip_duration = lambda x: x['match_matched_end_time'] - x['match_matched_start_time'],
                                                        origin_dist = lambda y: y.apply(lambda x: dist(x['match_from_stop_lat'], x['match_from_stop_lon'], x['o_stop_lat'], x['o_stop_lon']),axis=1),
                                                        next_origin_dist = lambda y: y.apply(lambda x: dist(x['match_to_stop_lat'], x['match_to_stop_lon'], x['next_o_stop_lat'], x['next_o_stop_lon']),axis=1)) \
                                        .sort_values(['cardNum','user_trip_id'])

        otp_buste_itineraries_summary = otp_buste_itineraries_summary[((otp_buste_itineraries_summary['trip_duration'] > pd.Timedelta('0s')) & 
                                                                                                                                        (otp_buste_itineraries_summary['trip_duration'] < pd.Timedelta('2h'))) &
                                                                                                                                ((otp_buste_itineraries_summary['start_diff'] > pd.Timedelta('0s')) &
                                                                                                                                        (otp_buste_itineraries_summary['start_diff'] < pd.Timedelta('1.5h')))] \
                                                                                .query('origin_dist < 0.1') \
                                                                                .query('next_origin_dist < 2.0')
        return otp_buste_itineraries_summary


In [12]:
def get_candidate_itineraries_penalty_score(otp_buste_itineraries_filtered):
        otp_buste_itineraries_penalty = otp_buste_itineraries_filtered \
                                                        .assign(penalty = lambda x: 2*x['start_diff'].dt.total_seconds() + x['trip_duration'].dt.total_seconds() + x['match_num_transfers']*10) \
                                                        [['cardNum','user_trip_id','itinerary_id','match_num_transfers','match_vehicle_boarding','next_origin_dist','origin_dist','start_diff','trip_duration','penalty']] \
                                                        .sort_values(['user_trip_id','penalty'], ascending=True)
        return otp_buste_itineraries_penalty


## Main

In [13]:
otp_suggestions_filepath = '/local/tarciso/data/otp-itineraries/otp_itineraries/2017_05_01_user_trips_100_otp_itineraries.csv'
user_trips_folderpath = '/local/tarciso/data/enhanced-buste/user_trips/'
bus_trips_folderpath = '/local/tarciso/data/enhanced-buste/bus_trips/'
gtfs_base_folderpath = '/local/tarciso/data/gtfs/'
output_folderpath = '/local/tarciso/data/test-odmat/'

In [14]:
file_date_str = otp_suggestions_filepath.split('/')[-1].split('_user_trips_')[0]
file_date = pd.to_datetime(file_date_str,format='%Y_%m_%d')
print "Processing File:", otp_suggestions_filepath

Processing File: /local/tarciso/data/otp-itineraries/otp_itineraries/2017_05_01_user_trips_100_otp_itineraries.csv


In [15]:
        # Extracting itinerary part name for later use
        itinerary_part_name = otp_suggestions_filepath.split('/')[-1].split('_')[5]
        # Read OTP Suggestions
        otp_suggestions_raw = pd.read_csv(otp_suggestions_filepath, parse_dates=['date','otp_start_time','otp_end_time'])

        if len(otp_suggestions_raw) == 0:
            print "Zero OTP suggestions found."
            print "Skipping next steps..."
            exit(0)


In [16]:
        # Prepare OTP data for analysis
        otp_suggestions = prepare_otp_data(otp_suggestions_raw)

        # Read stops data
        stops_filepath = gtfs_base_folderpath + os.sep + get_router_id(file_date) + os.sep + 'stops.txt'
        stops_df = pd.read_csv(stops_filepath)

        # Adding Parent Stop data to OTP Suggestions
        stops_parent_stations = stops_df[['stop_id','parent_station']]
        otp_suggestions = otp_suggestions.merge(stops_parent_stations.add_prefix('from_'),
                                                                                        on='from_stop_id',
                                                                                        how='left') \
                                        .merge(stops_parent_stations.add_prefix('to_'),
                                                on='to_stop_id',
                                                how='left')


In [17]:
        # Read Origin/Next-Origin Pairs for the same date
        trips_origins_filepath = user_trips_folderpath + os.sep + file_date_str + '_user_trips.csv'
        trips_origins = pd.read_csv(trips_origins_filepath,
                                                parse_dates=['o_boarding_datetime','o_gps_datetime','next_o_boarding_datetime','next_o_gps_datetime'])

        # Checking whether OTP and ticketing dates match
        dates_compatibility, otp_date, ticketing_date = compatible_dates(otp_suggestions,trips_origins)
        if not dates_compatibility:
                print "ERROR: OTP date", otp_date, "does not match Ticketing data", ticketing_date
                print "Skipping current day"
                exit(1)


In [18]:
        # Selecting trips for whom OTP suggestions were found
        selected_trips = trips_origins[trips_origins['o_boarding_id'].isin(otp_suggestions['user_trip_id'])]
        num_selected_trips = len(selected_trips)


In [24]:
        # Matching all kinds of boarding events to valid OTP suggestions
        itineraries_start = otp_suggestions.query('mode == \'BUS\'') \
                                .sort_values(['user_trip_id','itinerary_id','leg_id']) \
                                .groupby(['user_trip_id','itinerary_id']) \
                                .first() \
                                .reset_index()


In [25]:
        # Matching vehicle boarding origins
        matched_vehicle_boardings,num_matched_vehicle_boardings,vehicle_match_perc = match_vehicle_boardings(selected_trips,itineraries_start)
        print "Vehicle boardings with matching OTP suggestions: ", num_matched_vehicle_boardings, "(",vehicle_match_perc, "%)"


        # Matching terminal boarding origins
        matched_terminal_boardings,num_matched_terminal_boardings,terminal_matched_perc = match_terminal_boardings(selected_trips,itineraries_start)
        print "Terminal boardings with matching OTP suggestions: ", num_matched_terminal_boardings, "(", terminal_matched_perc, "%)"

        # Matching special case route 021 terminal boarding origins 
        num_terminal_021_boardings,matched_021_terminal_boardings,num_matched_021_terminal_boardings,terminal_021_match_perc = match_terminal_021_boardings(selected_trips,itineraries_start)
        if (num_terminal_021_boardings > 0):
                print "Line 021 Terminal boardings with matching OTP suggestions: ", num_matched_021_terminal_boardings, "(",terminal_021_match_perc, "%)"
        else:
                print "No Line 021 Terminal boardings found. Skipping matching."

        boarding_suggestions_matches = pd.concat([matched_vehicle_boardings,matched_021_terminal_boardings,matched_terminal_boardings])

        total_num_matches = num_matched_vehicle_boardings + num_matched_021_terminal_boardings + num_matched_terminal_boardings
        print "Total number of matches: ", total_num_matches, "(", 100*(total_num_matches/float(num_selected_trips)), "%)"

        if total_num_matches == 0:
            print "No match was found. Skipping next steps..."
            exit(0)


Vehicle boardings with matching OTP suggestions:  92 ( 80.701754386 %)
Terminal boardings with matching OTP suggestions:  56 ( 71.7948717949 %)
Line 021 Terminal boardings with matching OTP suggestions:  0 ( 0.0 %)
Total number of matches:  148 ( 74.0 %)


In [26]:
boarding_suggestions_matches.head()

,cardNum,o_boarding_id,o_boarding_datetime,o_gps_datetime,o_route,o_busCode,o_tripNum,o_stopPointId,o_stop_lat,o_stop_lon,...,leg_id,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,otp_duration_mins,from_parent_station,to_parent_station
0,3823073.0,28150.0,2017-05-01 13:36:25,2017-05-01 13:35:18,827,BC929,12.0,33788.0,-25.436303,-49.362221,...,1,2017-05-01 13:38:33,2017-05-01 13:45:00,BUS,827,33788.0,30993.0,6.450000,14508.0,14508.0
1,3823073.0,28150.0,2017-05-01 13:36:25,2017-05-01 13:35:18,827,BC929,12.0,33788.0,-25.436303,-49.362221,...,1,2017-05-01 14:28:17,2017-05-01 14:35:00,BUS,827,33788.0,30993.0,6.716667,14508.0,14508.0
2,2715287.0,28154.0,2017-05-01 13:36:28,2017-05-01 13:35:38,684,HA298,11.0,39378.0,-25.585230,-49.337210,...,1,2017-05-01 13:42:13,2017-05-01 14:18:00,BUS,684,39378.0,31053.0,35.783333,14494.0,14494.0
3,2715287.0,28154.0,2017-05-01 13:36:28,2017-05-01 13:35:38,684,HA298,11.0,39378.0,-25.585230,-49.337210,...,1,2017-05-01 13:57:13,2017-05-01 14:33:00,BUS,684,39378.0,31053.0,35.783333,14494.0,14494.0
4,2715287.0,28154.0,2017-05-01 13:36:28,2017-05-01 13:35:38,684,HA298,11.0,39378.0,-25.585230,-49.337210,...,1,2017-05-01 14:12:13,2017-05-01 14:48:00,BUS,684,39378.0,31053.0,35.783333,14494.0,14494.0


In [30]:
trips_origins.columns

Index([u'cardNum', u'o_boarding_id', u'o_boarding_datetime', u'o_gps_datetime',
       u'o_route', u'o_busCode', u'o_tripNum', u'o_stopPointId', u'o_stop_lat',
       u'o_stop_lon', u'next_o_boarding_id', u'next_o_boarding_datetime',
       u'next_o_gps_datetime', u'next_o_route', u'next_o_busCode',
       u'next_o_tripNum', u'next_o_stopPointId', u'next_o_stop_lat',
       u'next_o_stop_lon', u'boardings_timediff', u'dist_between_origins'],
      dtype='object')

In [31]:
boarding_suggestions_matches.columns

Index([u'cardNum', u'o_boarding_id', u'o_boarding_datetime', u'o_gps_datetime',
       u'o_route', u'o_busCode', u'o_tripNum', u'o_stopPointId', u'o_stop_lat',
       u'o_stop_lon', u'next_o_boarding_id', u'next_o_boarding_datetime',
       u'next_o_gps_datetime', u'next_o_route', u'next_o_busCode',
       u'next_o_tripNum', u'next_o_stopPointId', u'next_o_stop_lat',
       u'next_o_stop_lon', u'boardings_timediff', u'dist_between_origins',
       u'user_trip_id', u'itinerary_id', u'date', u'leg_id', u'otp_start_time',
       u'otp_end_time', u'mode', u'route', u'from_stop_id', u'to_stop_id',
       u'otp_duration_mins', u'from_parent_station', u'to_parent_station'],
      dtype='object')

In [59]:
def get_otp_matched_legs(boarding_suggestions_matches_itin,otp_suggestions):
        otp_legs_suggestions_matches = boarding_suggestions_matches_itin \
                                                                        .merge(otp_suggestions,
                                                                                        left_on=['o_boarding_id','itinerary_id'],
                                                                                        right_on=['user_trip_id','itinerary_id'],
                                                                                        how='inner')
        otp_legs_suggestions_matches['first_vehicle_boarding'] = np.where((otp_legs_suggestions_matches['mode'] == "BUS") & 
                                                                          (np.logical_not(otp_legs_suggestions_matches['o_busCode'].str.isdigit()) & 
                                                                           (otp_legs_suggestions_matches['o_route'] == otp_legs_suggestions_matches['route'])),
                                                                          True,
                                                                          False) #\
                                                                        #.filter(np.append(otp_suggestions.columns.values,['first_vehicle_boarding','o_busCode','o_tripNum','o_boarding_datetime']))
        return otp_legs_suggestions_matches


## Working with Vehicle Boardings

In [64]:
        # Add OTP extra origin/next-origin pairs to final dataset
        vehicle_boarding_matched_itins = matched_vehicle_boardings.filter(np.append(trips_origins.columns.values,['itinerary_id']))
        vehicle_otp_filtered_legs = get_otp_matched_legs(vehicle_boarding_matched_itins,otp_suggestions)
        vehicle_otp_filtered_bus_legs = get_otp_matched_legs(vehicle_boarding_matched_itins,otp_suggestions).query('mode == \'BUS\'')
        vehicle_otp_filtered_walk_legs = get_otp_matched_legs(vehicle_boarding_matched_itins,otp_suggestions).query('mode == \'WALK\'')


In [65]:
vehicle_otp_filtered_legs.head()

,cardNum,o_boarding_id,o_boarding_datetime,o_gps_datetime,o_route,o_busCode,o_tripNum,o_stopPointId,o_stop_lat,o_stop_lon,...,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,otp_duration_mins,from_parent_station,to_parent_station,first_vehicle_boarding
0,3823073.0,28150.0,2017-05-01 13:36:25,2017-05-01 13:35:18,827,BC929,12.0,33788.0,-25.436303,-49.362221,...,2017-05-01 13:38:33,2017-05-01 13:45:00,BUS,827,33788.0,30993.0,6.450000,NaN,14508.0,True
1,3823073.0,28150.0,2017-05-01 13:36:25,2017-05-01 13:35:18,827,BC929,12.0,33788.0,-25.436303,-49.362221,...,2017-05-01 13:45:00,2017-05-01 13:45:52,WALK,nan,NaN,NaN,0.866667,NaN,NaN,False
2,3823073.0,28150.0,2017-05-01 13:36:25,2017-05-01 13:35:18,827,BC929,12.0,33788.0,-25.436303,-49.362221,...,2017-05-01 13:51:00,2017-05-01 14:01:59,BUS,303,26195.0,25753.0,10.983333,14508.0,41756.0,False
3,3823073.0,28150.0,2017-05-01 13:36:25,2017-05-01 13:35:18,827,BC929,12.0,33788.0,-25.436303,-49.362221,...,2017-05-01 14:02:00,2017-05-01 14:09:32,WALK,nan,NaN,NaN,7.533333,NaN,NaN,False
4,3823073.0,28150.0,2017-05-01 13:36:25,2017-05-01 13:35:18,827,BC929,12.0,33788.0,-25.436303,-49.362221,...,2017-05-01 14:28:17,2017-05-01 14:35:00,BUS,827,33788.0,30993.0,6.716667,NaN,14508.0,True


In [66]:
vehicle_otp_filtered_legs.dtypes

cardNum                            float64
o_boarding_id                      float64
o_boarding_datetime         datetime64[ns]
o_gps_datetime              datetime64[ns]
o_route                             object
o_busCode                           object
o_tripNum                          float64
o_stopPointId                      float64
o_stop_lat                         float64
o_stop_lon                         float64
next_o_boarding_id                 float64
next_o_boarding_datetime    datetime64[ns]
next_o_gps_datetime         datetime64[ns]
next_o_route                        object
next_o_busCode                      object
next_o_tripNum                     float64
next_o_stopPointId                 float64
next_o_stop_lat                    float64
next_o_stop_lon                    float64
boardings_timediff                  object
dist_between_origins               float64
itinerary_id                         int64
date                        datetime64[ns]
user_trip_i

In [69]:
        # Find OTP Suggested Itineraries in BUSTE Data
        bus_trips_filepath = bus_trips_folderpath + os.sep + file_date_str + '_bus_trips.csv'
        bus_trips = pd.read_csv(bus_trips_filepath, dtype={'route': object},parse_dates=['gps_datetime']) \
                                        .sort_values(['route','busCode','tripNum','gps_datetime']) \
                                        .assign(route = lambda x: x['route'].astype(str).str.replace("\.0",'').str.zfill(3))


In [72]:
bus_trips.head()

,route,busCode,shapeId,tripNum,stopPointId,gps_datetime,distanceTraveledShape,stop_lat,stop_lon,parent_station
29435,010,BB303,1708.0,1.0,33157.0,2017-05-01 06:28:07,537.974,-25.410517,-49.276479,NaN
29436,010,BB303,1708.0,1.0,33159.0,2017-05-01 06:30:38,1141.061,-25.411726,-49.270902,NaN
29437,010,BB303,1708.0,1.0,33158.0,2017-05-01 06:31:40,1624.751,-25.415285,-49.270134,NaN
29438,010,BB303,1708.0,1.0,30150.0,2017-05-01 06:32:26,1972.077,-25.416733,-49.267863,NaN
29439,010,BB303,1708.0,1.0,28637.0,2017-05-01 06:33:11,2378.349,-25.414184,-49.265917,NaN


In [73]:
def match_vehicle_boardings_otp_legs_start_to_buste(otp_filtered_legs,bus_trips):
        otp_legs_buste_start = otp_filtered_legs \
                                                        .merge(bus_trips,
                                                                         left_on=['route','o_busCode','o_tripNum','from_stop_id'],
                                                                         right_on=['route','busCode','tripNum','stopPointId'],
                                                                         how='inner') \
                                                        .assign(otp_buste_start_timediff =
                                                                lambda x: np.absolute(x['gps_datetime'] - x['otp_start_time'])) \
                                                        .filter(['user_trip_id','first_vehicle_boarding','itinerary_id','leg_id', 'mode','route','busCode',
                                                                 'o_busCode','tripNum','o_tripNum','from_stop_id','otp_start_time','gps_datetime',
                                                                 'o_boarding_datetime','otp_buste_start_timediff','to_stop_id','otp_end_time', 'otp_duration_mins']) \
                                                        .sort_values(['user_trip_id','itinerary_id','leg_id','otp_buste_start_timediff']) \
                                                        .rename(index=str, columns={'to_stop_id':'stopPointId', 'gps_datetime':'matched_start_time'})

        otp_legs_buste_start = otp_legs_buste_start[otp_legs_buste_start['otp_buste_start_timediff'] < pd.Timedelta('60min')]
        return otp_legs_buste_start


In [74]:
        # Find legs candidate match start point
        otp_legs_buste_start = match_vehicle_boardings_otp_legs_start_to_buste(vehicle_otp_filtered_bus_legs,bus_trips)


In [77]:
otp_legs_buste_start.filter(np.append(bus_trips.columns,otp_suggestions.columns)).head()

,route,busCode,tripNum,stopPointId,user_trip_id,itinerary_id,leg_id,otp_start_time,otp_end_time,mode,route,from_stop_id,otp_duration_mins
0,827,BC929,12.0,30993.0,28150.0,1,1,2017-05-01 13:38:33,2017-05-01 13:45:00,BUS,827,33788.0,6.450000
1,827,BC929,12.0,30993.0,28150.0,1,1,2017-05-01 13:38:33,2017-05-01 13:45:00,BUS,827,33788.0,6.450000
2,827,BC929,12.0,30993.0,28150.0,1,1,2017-05-01 13:38:33,2017-05-01 13:45:00,BUS,827,33788.0,6.450000
3,827,BC929,12.0,30993.0,28150.0,1,1,2017-05-01 13:38:33,2017-05-01 13:45:00,BUS,827,33788.0,6.450000
4,827,BC929,12.0,30993.0,28150.0,8,1,2017-05-01 14:28:17,2017-05-01 14:35:00,BUS,827,33788.0,6.716667


In [8]:
def match_otp_legs_end_to_buste(otp_filtered_legs,bus_trips):
        otp_legs_buste = otp_legs_buste_start \
                                .merge(bus_trips,
                                                 on=['route','busCode','tripNum','stopPointId'],
                                                 how='inner') \
                                .assign(otp_buste_end_timediff =
                                                        lambda x: np.absolute(x['gps_datetime'] - x['otp_end_time'])) \
                                .rename(index=str, columns={'stopPointId':'to_stop_id', 'gps_datetime':'matched_end_time'}) \
                                .assign(matched_leg_duration_mins = lambda x: x['matched_end_time'] - x['matched_start_time'],
                                                boarding_matched_start_timediff =
                                                        lambda x: np.absolute(x['o_boarding_datetime'] - x['matched_start_time'])) \
                                .query('matched_end_time > matched_start_time') \
                                .filter(['user_trip_id','first_vehicle_boarding','itinerary_id','leg_id','mode','route','busCode',
                                                 'o_busCode','tripNum','o_tripNum','from_stop_id','otp_start_time',
                                                 'matched_start_time','o_boarding_datetime','otp_buste_start_timediff',
                                                 'to_stop_id','otp_end_time','matched_end_time','otp_buste_end_timediff',
                                                 'boarding_matched_start_timediff', 'otp_duration_mins','matched_leg_duration_mins']) \
                                .sort_values(['user_trip_id','itinerary_id','leg_id','otp_buste_end_timediff'])

        otp_legs_buste = otp_legs_buste[otp_legs_buste['otp_buste_end_timediff'] < pd.Timedelta('60min')]
        return otp_legs_buste
